In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np 
import torch
import pickle
import pandas as pd
from visualization import quantile, process_data, plot_subplots_concat, plot_subplots_compare
import os
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['svg.fonttype'] = 'none'

#### CDSTN only muli-epochs

In [ ]:
# unnormalization = True
# device = torch.device('cuda:1')

# nsample_CDSTN = 100 # number of generated sample
# foldername_CDSTN = "PeMS7_V_228_20230608_103057_missing_pattern(RSM)_misssing_rate(0.1)"
# max_epoch = 560

# dataind_CDSTN = 16

# subplots_ind_ls = [2, 6, 10, 11,12,15,20,21,22]
# ncols = 3
# nrows = (len(subplots_ind_ls) + ncols - 1) // ncols

# path = './save/' + foldername_CDSTN + '/meanstd.pk'
# with open(path, 'rb') as f:
#     train_mean_CDSTN,train_std_CDSTN = pickle.load(f)

# figs_path = './save/' + foldername_CDSTN + "/only_CDSTN_figs/"

# os.makedirs(figs_path, exist_ok=True)

# # read the file names of all generated samples in .pk format
# avai_gen_samples_files = [f for f in os.listdir("./save/"+foldername_CDSTN+"/") if f.startswith("generated_outputs_nsample")]
# avai_file_ls = []
# # Iterate through each file name, and store available epoch numbers
# for file_name in avai_gen_samples_files:
#     # Extract the epoch number    
#     avai_file_ls.append(int(file_name.split("_epoch")[1].split(".pk")[0]))

# fig_count = 0

# for epoch in range(max_epoch):
#     if epoch not in avai_file_ls:
#         continue

#     path_CDSTN = './save/' + foldername_CDSTN + '/generated_outputs_nsample' + str(nsample_CDSTN) + '_epoch' + str(epoch) + '.pk'

#     with open(path_CDSTN, 'rb') as f:
#         samples_CDSTN,all_target_CDSTN,all_evalpoint_CDSTN,all_observed_CDSTN,_,_,_ = pickle.load( f)

#     # transfer to another GPU
#     samples_CDSTN = samples_CDSTN.to(device)
#     all_target_CDSTN = all_target_CDSTN.to(device)
#     all_evalpoint_CDSTN = all_evalpoint_CDSTN.to(device)
#     all_observed_CDSTN = all_observed_CDSTN.to(device)

#     (
#         samples_CDSTN,
#         K, 
#         L_CDSTN, 
#         all_target_np_CDSTN, 
#         all_given_np_CDSTN, 
#         all_evalpoint_np_CDSTN
#     ) = process_data(
#         samples_CDSTN,
#         train_mean_CDSTN, 
#         train_std_CDSTN, 
#         all_target_CDSTN, 
#         all_evalpoint_CDSTN, 
#         all_observed_CDSTN, 
#         unnormalization)

#     quantiles_imp_CDSTN = quantile(samples_CDSTN, all_target_np_CDSTN, all_given_np_CDSTN)

#     fig_count += 1

#     plot_subplots_concat(
#         nrows, 
#         ncols, 
#         subplots_ind_ls, 
#         L_CDSTN, 
#         dataind_CDSTN, 
#         quantiles_imp_CDSTN, 
#         all_target_np_CDSTN, 
#         all_evalpoint_np_CDSTN, 
#         figs_path, 
#         epoch,
#         fig_count
#         );

#### Comparison CDSTN and SCDI

In [2]:
unnormalization = True
device = torch.device('cuda:1')

nsample_CDSTN = 100 # number of generated sample
foldername_CDSTN = "PeMS7_V_228_20230608_103057_missing_pattern(RSM)_misssing_rate(0.1)"
max_epoch = 560

dataind_CDSTN = 16

subplots_ind_ls = [2, 6, 10, 11,12,15,20,21,22]
ncols = 3
nrows = (len(subplots_ind_ls) + ncols - 1) // ncols

path = './save/' + foldername_CDSTN + '/meanstd.pk'
with open(path, 'rb') as f:
    train_mean_CDSTN,train_std_CDSTN = pickle.load(f)

figs_path = './save/' + foldername_CDSTN + "/model_comparison_figs/"

os.makedirs(figs_path, exist_ok=True)


foldername_CSDI = "PeMS7_V_228_20230607_155333_missing_pattern(RSM)_misssing_rate(0.1)"

dataind_CSDI = 0

path = './save_csdi/' + foldername_CSDI + '/meanstd.pk'
with open(path, 'rb') as f:
    train_mean_CSDI,train_std_CSDI = pickle.load(f)

# read the file names of all generated samples in .pk format
avai_gen_samples_files = [f for f in os.listdir("./save_csdi/"+foldername_CSDI+"/") if f.startswith("generated_outputs_nsample")]
avai_file_ls = []

# Iterate through each file name, and store available epoch numbers
for file_name in avai_gen_samples_files:
    # Extract the epoch number    
    avai_file_ls.append(int(file_name.split("_epoch")[1].split(".pk")[0]))

fig_count = 0

for epoch in range(max_epoch):
    if epoch not in avai_file_ls:
        continue

    path_CDSTN = './save/' + foldername_CDSTN + '/generated_outputs_nsample' + str(nsample_CDSTN) + '_epoch' + str(epoch) + '.pk'

    with open(path_CDSTN, 'rb') as f:
        samples_CDSTN,all_target_CDSTN,all_evalpoint_CDSTN,all_observed_CDSTN,_,_,_ = pickle.load( f)

    # transfer to another GPU
    samples_CDSTN = samples_CDSTN.to(device)
    all_target_CDSTN = all_target_CDSTN.to(device)
    all_evalpoint_CDSTN = all_evalpoint_CDSTN.to(device)
    all_observed_CDSTN = all_observed_CDSTN.to(device)

    (
        samples_CDSTN,
        K, 
        L_CDSTN, 
        all_target_np_CDSTN, 
        all_given_np_CDSTN,
        all_evalpoint_np_CDSTN
    )  = process_data(
        samples_CDSTN,
        train_mean_CDSTN, 
        train_std_CDSTN, 
        all_target_CDSTN, 
        all_evalpoint_CDSTN, 
        all_observed_CDSTN, 
        unnormalization)
    
    zip_data_CDSTN =     (
        samples_CDSTN,
        K, 
        L_CDSTN, 
        all_target_np_CDSTN, 
        all_evalpoint_np_CDSTN
    )

    quantiles_imp_CDSTN = quantile(samples_CDSTN, all_target_np_CDSTN, all_given_np_CDSTN)    



    # reading csdi samples
    path_CSDI = './save_csdi/' + foldername_CSDI + '/generated_outputs_nsample' + str(nsample_CDSTN) + '_epoch' + str(epoch) + '.pk'

    with open(path_CSDI, 'rb') as f:
        samples_CSDI,all_target_CSDI,all_evalpoint_CSDI,all_observed_CSDI,_,_,_ = pickle.load( f)

    # transfer to another GPU
    samples_CSDI = samples_CSDI.to(device)
    all_target_CSDI = all_target_CSDI.to(device)
    all_evalpoint_CSDI = all_evalpoint_CSDI.to(device)
    all_observed_CSDI = all_observed_CSDI.to(device)

    (
        samples_CSDI,
        K, 
        L_CSDI, 
        all_target_np_CSDI, 
        all_given_np_CSDI, 
        all_evalpoint_np_CSDI
    ) = process_data(
        samples_CSDI,
        train_mean_CSDI, 
        train_std_CSDI, 
        all_target_CSDI, 
        all_evalpoint_CSDI, 
        all_observed_CSDI, 
        unnormalization)
    
    zip_data_CSDI = (
        samples_CSDI,
        K, 
        L_CSDI, 
        all_target_np_CSDI, 
        all_evalpoint_np_CSDI
    )

    quantiles_imp_CSDI = quantile(samples_CSDI, all_target_np_CSDI, all_given_np_CSDI)    

    fig_count += 1
    plot_subplots_compare(
        nrows,
        ncols,
        subplots_ind_ls,
        dataind_CDSTN,
        dataind_CSDI,
        quantiles_imp_CDSTN,
        quantiles_imp_CSDI,
        zip_data_CDSTN,
        zip_data_CSDI,
        figs_path, 
        epoch,
        fig_count
    );